# Fit PDF profile - Si standard from NOMAD

The input datafiles were extracted from `2017A_Cryostat_Calibration.ddp` (from NOMAD at SNS) using pdfgui: five .fgr files corresponding to different r-ranges and different sets of refined parameters.  

The objective is to compare the values of the fitted parameters with those from pdfgui. For the parameters to be refined, we chose to use the same initial guesses as in pdfgui.  

Note that the CIF file used to refine the data in the notebook is different from the one used in pdfgui. That's why the initial model might not correspond to the best initial guess to match the observed $G(r)$ in some cases.

In [ ]:
import os

import pandas as pd

from easyscience.fitting.fitter import Fitter
from easyDiffractionLib import Phases
from easyDiffractionLib.interface import InterfaceFactory as Calculator
from easyDiffractionLib.Profiles.P1D import PDFParameters
from easyDiffractionLib.Interfaces.pdffit2 import readGRData
from easyDiffractionLib.Jobs import Powder1DTOF
from easyscience.Fitting.Constraints import ObjConstraint

import matplotlib.pyplot as plt
import py3Dmol

%matplotlib widget

## Define functions for plotting

In [ ]:
def plot_ini_config(x, Gobs, Gsim, title):
    """
    Plots of the observed data and initial model
    The top plot displays the observed and simulated curves
    The bottom plot displays the difference between observed 
    and simulated curves.

    Parameters
    ----------

    x: 1D numpy.array, x-values for the plot.
       We assume that both curves use the same x-values.
        
    Gobs: 1D numpy.array, y-values of the observed / measured 
           and reduced data

    Gsim: 1D numpy.array, y-values of the initial model 

    title: string, Title of the plot
    
    Returns
    ----------
    None

    """
    Gdiff = Gobs - Gsim
    
    fig, ax = plt.subplots(2, 1, 
                           sharex=True, 
                           figsize=(12, 12), 
                           gridspec_kw={'height_ratios': [2, 1]})
    fig.suptitle(title)
    ax[0].plot(x, Gobs, label='G(r) Data', color='steelblue', marker='o')
    ax[0].plot(x, Gsim, label='G(r) Sim', color='orangered', linewidth=1.2)

    ax[1].plot(x, Gdiff, color='grey', linewidth=1.2)
    ax[1].set_xlabel('r (Å)')
    ax[0].set_ylabel(r"$G(r)\ (Å^{-2})$")
    ax[1].set_ylabel(r"$G(r)\ Diff\ (Å^{-2})$")

    ax[0].legend()
    ax[0].grid(); ax[1].grid()

    return


def plot_final_config(x, Gobs, Gsim, Gfit, title):
    """
    Plot of the observed data, initial model and fitting results

    Parameters
    ----------

    x: 1D numpy.array, x-values for the plot.
       We assume that all 3 curves use the same x-values.
        
    Gobs: 1D numpy.array, y-values of the observed / measured 
           and reduced data

    Gsim: 1D numpy.array, y-values of the initial model 

    Gfit: 1D numpy.array, y-values of the fitting result

    title: string, Title of the plot
    
    Returns
    ----------
    None

    """
    fig, ax = plt.subplots(2, 1, 
                         sharex=True, 
                         figsize=(12, 12), 
                         gridspec_kw={'height_ratios': [2, 1]})
    fig.suptitle(title)

    Gdiff = Gobs - Gfit

    ax[0].plot(x, Gobs, label='G(r) Data', 
               linestyle='', marker='o', markeredgecolor='steelblue', markerfacecolor='none')
    ax[0].plot(x, Gsim, label='G(r) Sim', linestyle='dashed', color='green', linewidth=1.2)
    ax[0].plot(x, Gfit, label='G(r) Fit', color='orangered', linewidth=1.2)

    ax[1].plot(x, Gdiff, color='grey', linewidth=1.2)
    ax[1].set_xlabel('r (Å)')
    ax[0].set_ylabel(r"$G(r)\ (Å^{-2})$")
    ax[1].set_ylabel(r"$G(r)\ Diff\ (Å^{-2})$")

    ax[0].legend()
    ax[0].grid(); ax[1].grid()
    
    return


def plot_comparison_refined_params(df, title):
    """
    Plot relative difference of refined values between pdfgui and EasyDiffraction

    Parameters
    ----------

    df: pandas.DataFrame, contains initial and refined parameters for pdfgui and EasyDiffraction
    
    title: string, Title of the plot
    
    Returns
    ----------
    None
    
    """
    comparison_params = {}
    for param in df_case1.index:
        comparison_params[param] =  abs(df_case1.at[param,'pdfgui_fit'] - df_case1.at[param,'ezd_fit']) / df_case1.at[param,'pdfgui_fit']

    names = list(comparison_params.keys())
    values = list(comparison_params.values())

    fig, ax = plt.subplots()
    ax.scatter(names, values , marker='x')
    ax.set_xticklabels(names, rotation = 45)
    ax.set_ylabel(r"$\frac{|param_{pdfgui}- param_{ezd}|}{param_{pdfgui}}$", fontsize=16)
    fig.suptitle(f'{title}:\nrelative difference of refined parameters\n between pdfgui and EasyDiffraction (ezd)');
    ax.grid()
    plt.tight_layout()
    
    return

## Path to CIF file
It will be loaded for each case in order to create the corresponding `Phases` component.

In [ ]:
cif_fname_Si = os.path.realpath('./data/4507226.cif')

## Check difference between .fgr files
These files contain reduced data in the format `r(A) G(r) d_r d_Gr Gdiff`.

In [ ]:
path2files = './data/'

data_Si_100A = readGRData(path2files+'Si_calibration_cryostat_100A.fgr')

data_Si_20A = readGRData(path2files+'Si_calibration_cryostat_20A.fgr')

data_Si_20A_01Rmin = readGRData(path2files+'Si_calibration_cryostat_20A_01Rmin.fgr')

data_Si_50A = readGRData(path2files+'Si_calibration_cryostat_50A.fgr')

data_Si_50A_01Rmin = readGRData(path2files+'Si_calibration_cryostat_50A_01Rmin.fgr')

In [ ]:
fig = plt.figure(figsize=(10,10))

plt.plot(data_Si_100A[:, 0], 
         data_Si_100A[:, 1], 
         linestyle='', 
         marker='.', 
         label=r'100 Å', 
         markersize=2)

plt.plot(data_Si_50A[:, 0], 
         data_Si_50A[:, 1], 
         label=r'50 Å', 
         linewidth=2)

plt.plot(data_Si_50A_01Rmin[:, 0], 
         data_Si_50A_01Rmin[:, 1], 
         linestyle='', 
         marker='.', 
         label=r'50 Å 01$R_{min}$', 
         markersize=2)

plt.plot(data_Si_20A[:, 0], 
         data_Si_20A[:, 1], 
         label=r'20 Å', 
         linewidth=2)

plt.plot(data_Si_20A_01Rmin[:, 0], 
         data_Si_20A_01Rmin[:, 1], 
         linestyle='', 
         marker='.', 
         label=r'20 Å 01$R_{min}$', 
         markersize=2)

plt.xlabel(r"$r (Å)$")
plt.ylabel(r"$G (Å^{-2})$")
plt.title('Fit of nickel to x-ray experimental PDF')
plt.legend()
plt.grid()

## Case 1
Information from pdfgui:

- File: `Si_Calibration_Cryostat_50A`   
- Fit range: 1.5 - 49.99 Angstrom
- Refined parameters with initial and refined values

| Name         | Initial value    | Refined value    |
|--------------|------------------|------------------|
| scale factor | 1.64773047742    | 1.64774086138    |
| delta2       | 3.83386044222    | 3.83389797886    |
| a=b=c        | 5.43323762429    | 5.43323744088    |
| uii          | 0.00817005319337 | 0.00817032613268 |
| Qdamp        | 0.0239248233329  | 0.0239255367627  |
| Qbroad       | 0.0195652517637  | 0.0195630501755  |

### Store results from pdfgui in pandas DataFrame

In [ ]:
df_case1 = pd.DataFrame(
    [['scale', 1.64773047742, 1.64774086138], 
     ['delta2', 3.83386044222, 3.83389797886], 
     ['length_a', 5.43323762429, 5.43323744088],
     ['Uiso', 0.00817005319337, 0.00817032613268], 
     ['Q_damp', 0.0239248233329, 0.0239255367627],
     ['Q_broad', 0.0195652517637, 0.0195630501755]],
    columns=['Parameters', 'pdfgui_ini', 'pdfgui_fit']).set_index('Parameters')
df_case1

### Define Phases for Case 1

In [ ]:
phases_Si1 = Phases.from_cif_file(cif_fname_Si)
phases_Si1

#### Visualize structure

In [ ]:
viewer = py3Dmol.view()

viewer.addModel(phases_Si1[0].cif,
                'cif',
                {'doAssembly': True,
                 'duplicateAssemblyAtoms': True,
                 'normalizeAssembly': True})

viewer.setStyle({'sphere': {'colorscheme': 'Jmol', 'scale': .2}, 
                 'stick': {'colorscheme': 'Jmol', 'radius': 0.1}})
viewer.addUnitCell()
viewer.replicateUnitCell(2, 2, 2)
viewer.zoomTo()

### Define Calculator for Case 1

In [ ]:
calculator_Si1 = Calculator()
calculator_Si1.switch("Pdffit2")

### Create Job

In [ ]:
job_Si1 = Powder1DTOF(
    'Si_Calibration_Cryostat_50A_simulated', 
    parameters=PDFParameters(), 
    phases=phases_Si1, 
    interface=calculator_Si1
)

fitter_Si1 = Fitter(job_Si1, calculator_Si1.fit_func)

#### Set parameters

In [ ]:
parameters1 = job_Si1.parameters

parameters1.get_parameters()

In [ ]:
# parameters to be refined
parameters1.delta2 = df_case1['pdfgui_ini']['delta2'] 
parameters1.qdamp = df_case1['pdfgui_ini']['Q_damp']  
parameters1.qbroad = df_case1['pdfgui_ini']['Q_broad'] 

# set values for fixed parameters using metadata in header of .fgr file
parameters1.qmax = 31.4 

parameters1.delta1 = 0.0

# check modified parameters
parameters1.get_parameters()

In [ ]:
phase1_patterns1 = job_Si1.phases[0]
phase1_patterns1.get_parameters()

In [ ]:
phases_Si1[0].spacegroup

In [ ]:
phase1_patterns1.cell.length_a = df_case1['pdfgui_ini']['length_a']  

phase1_patterns1.atoms[0].adp.Uiso = df_case1['pdfgui_ini']['Uiso']

phases_Si1[0].scale = df_case1['pdfgui_ini']['scale']

# Check modified parameters
phase1_patterns1.get_parameters()

In [ ]:
# Add initial data used in EasyDiffraction to pandas DataFrame
df_case1['ezd_ini'] = [phases_Si1[0].scale.value, 
                       parameters1.delta2.value, 
                       phase1_patterns1.cell.length_a.value, 
                       phase1_patterns1.atoms[0].adp.Uiso.value, 
                       parameters1.qdamp.value, 
                       parameters1.qbroad.value]

#### Plot initial profile and compare with observed data

In [ ]:
x_data_Si_50A = data_Si_50A[:, 0]
Gobs_Si_50A = data_Si_50A[:, 1]
Gsim_Si_50A = calculator_Si1.fit_func(x_data_Si_50A)

plot_ini_config(x_data_Si_50A, 
                Gobs_Si_50A, 
                Gsim_Si_50A, 
                r'Si 50 Å to neutron experimental PDF')

#### Select parameters to optimize

For case 1, they are
- scale factor
- delta2
- a=b=c
- uii
- Qdamp
- Qbroad

In [ ]:
phase1_patterns1.cell.length_a.fixed = False

phase1_patterns1.atoms[0].adp.Uiso.fixed = False

phases_Si1[0].scale.fixed = False

parameters1.qdamp.fixed = False
parameters1.qbroad.fixed = False
parameters1.delta2.fixed = False

#### Fit

In [ ]:
result_Si1 = fitter_Si1.fit(x_data_Si_50A, data_Si_50A[:, 1], 
                    method='least_squares', 
                    minimizer_kwargs={'diff_step': 1e-5, 
                                      'verbose': 2})

#### Print and plot results

In [ ]:
# Add refined values to pandas DataFrame
df_case1['ezd_fit'] = pd.Series(dtype='float64')

for i in range(len(job_Si1.get_fit_parameters())):
    df_case1['ezd_fit'][job_Si1.get_fit_parameters()[i].name] = job_Si1.get_fit_parameters()[i].value
df_case1

In [ ]:
# Plot relative difference of refined values between pdfgui and EasyDiffraction
plot_comparison_refined_params(df_case1, 'Case 1')

In [ ]:
print(f"The fit has been successful: {result_Si1.success}")  
print(f"The goodness of fit (chi2) is: {result_Si1.reduced_chi}")
print("The optimized parameters are:")

for param in job_Si1.get_fit_parameters():
    print(f"{param.name}: {param.value} +\- {param.variance} {param.unit}") 

In [ ]:
# obtain data from PdfFit calculator object
Gfit_Si_50A = calculator_Si1.fit_func(x_data_Si_50A)

plot_final_config(x_data_Si_50A, 
                  Gobs_Si_50A, 
                  Gsim_Si_50A, 
                  Gfit_Si_50A, 
                  r'Si 50 Å to neutron experimental PDF')

## Case 2
- File: Si_Calibration_Cryostat_20A
- Fit range: 1.5 - 20 Angstrom
- Refined parameters with initial and refined values

| Name         | Initial value    | Refined value   |
|--------------|------------------|-----------------|
| scale factor | 1.66361225617    | 1.66360976627   |
| delta2       | 3.68122318362    | 3.68123080818   |
| a=b=c        | 5.43487259837    | 5.43487255753   |
| uii          | 0.00788328880945 | 0.0078832632553 |

### Store results from pdfgui in pandas DataFrame

In [ ]:
df_case2 = pd.DataFrame(
    [['scale', 1.66361225617, 1.66360976627], 
     ['delta2', 3.68122318362, 3.68123080818], 
     ['length_a', 5.43487259837, 5.43487255753],
     ['Uiso', 0.00788328880945, 0.0078832632553]], 
    columns=['Parameters', 'pdfgui_ini', 'pdfgui_fit']).set_index('Parameters')
df_case2

### Define Phases for Case 2

In [ ]:
phases_Si2 = Phases.from_cif_file(cif_fname_Si)
phases_Si2

### Define Calculator for Case 2

In [ ]:
calculator_Si2 = Calculator()
calculator_Si2.switch("Pdffit2")

### Create Job

In [ ]:
job_Si2 = Powder1DTOF(
    'Si_Calibration_Cryostat_20A_simulated', 
    parameters=PDFParameters(), 
    phases=phases_Si2, 
    interface=calculator_Si2
)

fitter_Si2 = Fitter(job_Si2, calculator_Si2.fit_func)

#### Set parameters

In [ ]:
parameters2 = job_Si2.parameters

parameters2.get_parameters()

In [ ]:
# parameters to be refined
parameters2.delta2 = df_case2['pdfgui_ini']['delta2']

# set values for fixed parameters using metadata in header of .fgr file
parameters2.qdamp = 0.024  
parameters2.qbroad = 0.02  
parameters2.qmax =  31.4 

parameters2.delta1 = 0.0

In [ ]:
phase1_patterns2 = job_Si2.phases[0]

In [ ]:
phase1_patterns2.cell.length_a = df_case2['pdfgui_ini']['length_a'] 

phase1_patterns2.atoms[0].adp.Uiso = df_case2['pdfgui_ini']['Uiso']

phases_Si2[0].scale =  df_case2['pdfgui_ini']['scale']

In [ ]:
# Check modified parameters
phase1_patterns2.get_parameters()

In [ ]:
# Add initial data used in EasyDiffraction to pandas DataFrame
df_case2['ezd_ini'] = [phases_Si2[0].scale.value, 
                       parameters2.delta2.value, 
                       phase1_patterns2.cell.length_a.value, 
                       phase1_patterns2.atoms[0].adp.Uiso.value]

#### Calculate and plot initial profile and compare with observed data

In [ ]:
x_data_Si_20A = data_Si_20A[:, 0]
Gobs_Si_20A = data_Si_20A[:, 1]
Gsim_Si_20A = calculator_Si2.fit_func(x_data_Si_20A)

plot_ini_config(x_data_Si_20A, 
                Gobs_Si_20A, 
                Gsim_Si_20A, 
                r'Si 20 Å to neutron experimental PDF')

#### Select parameters to optimize

For case 2, they are

- scale factor
- a=b=c
- uii
- delta2

In [ ]:
phase1_patterns2.atoms[0].adp.Uiso.fixed = False

phases_Si2[0].scale.fixed = False

phase1_patterns2.cell.length_a.fixed = False

parameters2.delta2.fixed = False

In [ ]:
df_case2['ezd_ini'] = [phases_Si2[0].scale.value, 
                       parameters2.delta2.value, 
                       phase1_patterns2.cell.length_a.value, 
                       phase1_patterns2.atoms[0].adp.Uiso.value
                      ]
df_case2

#### Fit

In [ ]:
result_Si2 = fitter_Si2.fit(x_data_Si_20A, data_Si_20A[:, 1], 
                    method='least_squares', 
                    minimizer_kwargs={'diff_step': 1e-5, 'verbose': 2})

#### Print and plot results

In [ ]:
# Add refined values to pandas DataFrame
df_case2['ezd_fit'] = pd.Series(dtype='float64')

for i in range(len(job_Si2.get_fit_parameters())):
    df_case2['ezd_fit'][job_Si2.get_fit_parameters()[i].name] = job_Si2.get_fit_parameters()[i].value
df_case2

In [ ]:
# Plot relative difference of refined values between pdfgui and EasyDiffraction
plot_comparison_refined_params(df_case2, 'Case 2')

In [ ]:
print(f"The fit has been successful: {result_Si2.success}")  
print(f"The goodness of fit (chi2) is: {result_Si2.reduced_chi}")
print("The optimized parameters are:")
for param in job_Si2.get_fit_parameters():
    print(f"{param.name}: {param.value} +\- {param.variance} {param.unit}") 

| Symbol | Name         | Initial value    | Refined value   |
|--------|--------------|------------------|-----------------|
| @1     | scale factor | 1.66361225617    | 1.66360976627   |
| @2     | delta2       | 3.68122318362    | 3.68123080818   |
| @11    | a=b=c        | 5.43487259837    | 5.43487255753   |
| @21    | uii          | 0.00788328880945 | 0.0078832632553 |

In [ ]:
# obtain data from PdfFit calculator object
Gobs_Si_20A = data_Si_20A[:, 1]
Gfit_Si_20A = calculator_Si2.fit_func(x_data_Si_20A)

plot_final_config(x_data_Si_20A, 
                  Gobs_Si_20A, 
                  Gsim_Si_20A, 
                  Gfit_Si_20A, 
                  r'Si 20 Å to neutron experimental PDF')

## Case 3

- Si_Calibration_Cryostat_100A
- Fit range: 1.5-99.99 Angstrom
- Refined parameters with initial and refined values

| Name         | Initial value    | Refined value    |
|--------------|------------------|------------------|
| scale factor | 1.64773047742    | 1.59645132741    |
| delta2       | 3.83386044222    | 4.21474232159    |
| a=b=c        | 5.43323762429    | 5.43253113317    |
| uii          | 0.00817005319337 | 0.00927016592601 |
| Qdamp        | 0.024            | 0.021822511225   |
| Qbroad       | 0.02             | 0.0120657361893  |

### Store results from pdfgui in pandas DataFrame

In [ ]:
df_case3 = pd.DataFrame(
    [['scale', 1.64773047742, 1.59645132741], 
     ['delta2', 3.83386044222, 4.21474232159], 
     ['length_a', 5.43323762429, 5.43253113317],
     ['Uiso', 0.00817005319337, 0.00927016592601], 
     ['Q_damp', 0.024, 0.021822511225],
     ['Q_broad', 0.02, 0.0120657361893]],
    columns=['Parameters', 'pdfgui_ini', 'pdfgui_fit']).set_index('Parameters')
df_case3

### Define Phases for Case 3

In [ ]:
phases_Si3 = Phases.from_cif_file(cif_fname_Si)
phases_Si3

### Define Calculator for Case 1

In [ ]:
calculator_Si3 = Calculator()
calculator_Si3.switch("Pdffit2")

### Create Job

In [ ]:
job_Si3 = Powder1DTOF(
    'Si_Calibration_Cryostat_100A_simulated', 
    parameters=PDFParameters(), 
    phases=phases_Si3, 
    interface=calculator_Si3
)

fitter_Si3 = Fitter(job_Si3, calculator_Si3.fit_func)

#### Set parameters

In [ ]:
parameters3 = job_Si3.parameters

parameters3.get_parameters()

In [ ]:
# parameters to be refined
parameters3.delta2 = df_case3['pdfgui_ini']['delta2'] 
parameters3.qdamp = df_case3['pdfgui_ini']['Q_damp']
parameters3.qbroad = df_case3['pdfgui_ini']['Q_broad']

# set values for fixed parameters using metadata in header of .fgr file
parameters3.qmax = 31.4  

parameters3.delta1 = 0.0

In [ ]:
phase1_patterns3 = job_Si3.phases[0]
phase1_patterns3.get_parameters()

In [ ]:
phase1_patterns3.cell.length_a = df_case3['pdfgui_ini']['length_a'] 

phase1_patterns3.atoms[0].adp.Uiso = df_case3['pdfgui_ini']['Uiso'] 

phases_Si3[0].scale = df_case3['pdfgui_ini']['scale']

In [ ]:
# Add initial data used in EasyDiffraction to pandas DataFrame
df_case3['ezd_ini'] = [phases_Si3[0].scale.value, 
                       parameters3.delta2.value, 
                       phase1_patterns3.cell.length_a.value, 
                       phase1_patterns3.atoms[0].adp.Uiso.value, 
                       parameters3.qdamp.value, 
                       parameters3.qbroad.value]

#### Plot initial profile and compare with observed data

In [ ]:
x_data_Si_100A = data_Si_100A[:, 0]
Gobs_Si_100A = data_Si_100A[:, 1]
Gsim_Si_100A = calculator_Si1.fit_func(x_data_Si_100A)

plot_ini_config(x_data_Si_100A, Gobs_Si_100A, Gsim_Si_100A, r'Si 100 Å to neutron experimental PDF')

#### Select parameters to optimize

For case 3, they are
- scale factor
- delta2
- a=b=c
- uii
- Qdamp
- Qbroad

In [ ]:
phase1_patterns3.cell.length_a.fixed = False

phase1_patterns3.atoms[0].adp.Uiso.fixed = False

phases_Si3[0].scale.fixed = False

parameters3.qdamp.fixed = False
parameters3.qbroad.fixed = False
parameters3.delta2.fixed = False

#### Fit

In [ ]:
result_Si3 = fitter_Si3.fit(x_data_Si_100A, 
                            data_Si_100A[:, 1], 
                            method='least_squares', 
                            minimizer_kwargs={'diff_step': 1e-5, 
                                              'verbose': 2})

#### Print and plot results

In [ ]:
# Add refined values to pandas DataFrame
df_case3['ezd_fit'] = pd.Series(dtype='float64')

for i in range(len(job_Si3.get_fit_parameters())):
    df_case3['ezd_fit'][job_Si3.get_fit_parameters()[i].name] = job_Si3.get_fit_parameters()[i].value
df_case3

In [ ]:
# Plot relative difference of refined values between pdfgui and EasyDiffraction
plot_comparison_refined_params(df_case3, 'Case 3')

In [ ]:
print(f"The fit has been successful: {result_Si3.success}")  
print(f"The goodness of fit (chi2) is: {result_Si3.reduced_chi}")
print("The optimized parameters are:")

for param in job_Si3.get_fit_parameters():
    print(f"{param.name}: {param.value} +\- {param.variance} {param.unit}") 

In [ ]:
# obtain data from PdfFit calculator object
Gfit_Si_100A = calculator_Si3.fit_func(x_data_Si_100A)

plot_final_config(x_data_Si_100A,
                  Gobs_Si_100A,
                  Gsim_Si_100A,
                  Gfit_Si_100A,
                  r'Si 100 Å to neutron experimental PDF')

## Case 4

- Si_Calibration_Cryostat_50A_01Rmin
- Fit range: 0.1 - 49.99 Angstrom
- Refined parameters with initial and refined values

| Name         | Initial value    | Refined value    |
|--------------|------------------|------------------|
| scale factor | 1.64654880799    | 1.64654880799    |
| delta2       | 3.8378816337     | 3.8378816337     |
| a=b=c        | 5.43323692356    | 5.43323692356    |
| uii          | 0.00816331599398 | 0.00816331599398 |
| Qdamp        | 0.023900280307   | 0.023900280307   |
| Qbroad       | 0.0195887880295  | 0.0195887880296  |

### Store results from pdfgui in pandas DataFrame

In [ ]:
df_case4 = pd.DataFrame(
    [['scale', 1.64654880799, 1.64654880799], 
     ['delta2', 3.8378816337, 3.8378816337], 
     ['length_a', 5.43323692356, 5.43323692356],
     ['Uiso', 0.00816331599398, 0.00816331599398], 
     ['Q_damp', 0.023900280307, 0.023900280307],
     ['Q_broad', 0.0195887880295, 0.0195887880296]],
    columns=['Parameters', 'pdfgui_ini', 'pdfgui_fit']).set_index('Parameters')
df_case4

### Define Phases for Case 4

In [ ]:
phases_Si4 = Phases.from_cif_file(cif_fname_Si)
phases_Si4

### Define Calculator for Case 4

In [ ]:
calculator_Si4 = Calculator()
calculator_Si4.switch("Pdffit2")

### Create Job

In [ ]:
job_Si4 = Powder1DTOF(
    'Si_Calibration_Cryostat_100A_01Rmin_simulated', 
    parameters=PDFParameters(), 
    phases=phases_Si4, 
    interface=calculator_Si4
)

fitter_Si4 = Fitter(job_Si4, calculator_Si4.fit_func)

#### Set parameters

In [ ]:
parameters4 = job_Si4.parameters

parameters4.get_parameters()

In [ ]:
# parameters to be refined
parameters4.delta2 = df_case4['pdfgui_ini']['delta2']
parameters4.qdamp = df_case4['pdfgui_ini']['Q_damp'] 
parameters4.qbroad = df_case4['pdfgui_ini']['Q_broad']

# set values for fixed parameters using metadata in header of .fgr file
parameters4.qmax =  31.4

parameters4.delta1 = 0.0

In [ ]:
phase1_patterns4 = job_Si4.phases[0]
phase1_patterns4.get_parameters()

In [ ]:
phase1_patterns4.cell.length_a = df_case4['pdfgui_ini']['length_a'] 

phase1_patterns4.atoms[0].adp.Uiso = df_case4['pdfgui_ini']['Uiso'] 

phases_Si4[0].scale = df_case4['pdfgui_ini']['scale']

In [ ]:
# Add initial data used in EasyDiffraction to pandas DataFrame
df_case4['ezd_ini'] = [phases_Si4[0].scale.value, 
                       parameters4.delta2.value, 
                       phase1_patterns4.cell.length_a.value, 
                       phase1_patterns4.atoms[0].adp.Uiso.value, 
                       parameters4.qdamp.value, 
                       parameters4.qbroad.value]

#### Plot initial profile and compare with observed data

In [ ]:
x_data_Si_50A_01Rmin = data_Si_50A_01Rmin[:, 0]
Gobs_Si_50A_01Rmin = data_Si_50A_01Rmin[:, 1]
Gsim_Si_50A_01Rmin = calculator_Si4.fit_func(x_data_Si_50A_01Rmin)

plot_ini_config(x_data_Si_50A_01Rmin,
                Gobs_Si_50A_01Rmin,
                Gsim_Si_50A_01Rmin,
                r'Si 50 Å 01Rmin to neutron experimental PDF'
                )

#### Select parameters to optimize

For case 1, they are
- scale factor
- delta2
- a=b=c
- uii
- Qdamp
- Qbroad

In [ ]:
phase1_patterns4.cell.length_a.fixed = False

phase1_patterns4.atoms[0].adp.Uiso.fixed = False

phases_Si4[0].scale.fixed = False

parameters4.qdamp.fixed = False
parameters4.qbroad.fixed = False
parameters4.delta2.fixed = False

#### Fit

In [ ]:
result_Si4 = fitter_Si4.fit(x_data_Si_50A_01Rmin, data_Si_50A_01Rmin[:, 1], 
                    method='least_squares', 
                    minimizer_kwargs={'diff_step': 1e-5, 
                                      'verbose': 2})

#### Print and plot results

In [ ]:
# Add refined values to pandas DataFrame
df_case4['ezd_fit'] = pd.Series(dtype='float64')

for i in range(len(job_Si4.get_fit_parameters())):
    df_case4['ezd_fit'][job_Si4.get_fit_parameters()[i].name] = job_Si4.get_fit_parameters()[i].value
df_case4

In [ ]:
# Plot relative difference of refined values between pdfgui and EasyDiffraction
plot_comparison_refined_params(df_case4, 'Case 4')

In [ ]:
print(f"The fit has been successful: {result_Si4.success}")  
print(f"The goodness of fit (chi2) is: {result_Si4.reduced_chi}")
print("The optimized parameters are:")

for param in job_Si4.get_fit_parameters():
    print(f"{param.name}: {param.value} +\- {param.variance} {param.unit}") 

In [ ]:
# obtain data from PdfFit calculator object
Gfit_Si_50A_01Rmin = calculator_Si4.fit_func(x_data_Si_50A_01Rmin)

plot_final_config(x_data_Si_50A_01Rmin, 
                  Gobs_Si_50A_01Rmin,
                  Gsim_Si_50A_01Rmin,
                  Gfit_Si_50A_01Rmin,
                  r'Si 50 Å 01Rmin to neutron experimental PDF'
                 )

## Case 5
- Si_Calibation_Cryostat_20A_01Rmin
- Fit range: 0.1 - 20 Angstrom
- Refined parameters with initial and refined values

| Name         | Initial value    | Refined value    |
|--------------|------------------|------------------|
| scale factor | 1.66238320925    | 1.66238296494    |
| delta2       | 3.68476007538    | 3.68476092869    |
| a=b=c        | 5.43487069166    | 5.43487068759    |
| uii          | 0.00787662883607 | 0.00787662635073 |

### Store results from pdfgui in pandas DataFrame

In [ ]:
df_case5 = pd.DataFrame(
    [['scale', 1.66238320925, 1.66238296494], 
     ['delta2', 3.68476007538, 3.68476092869], 
     ['length_a', 5.43487069166, 5.43487068759],
     ['Uiso', 0.00787662883607, 0.00787662635073]],
    columns=['Parameters', 'pdfgui_ini', 'pdfgui_fit']).set_index('Parameters')
df_case5

### Create Phases and efine Calculator for Case 5

In [ ]:
phases_Si5 = Phases.from_cif_file(cif_fname_Si)
phases_Si5

In [ ]:
calculator_Si5 = Calculator()
calculator_Si5.switch("Pdffit2")

### Create Job

In [ ]:
job_Si5 = Powder1DTOF(
    'Si_Calibration_Cryostat_20A_01Rmin_simulated', 
    parameters=PDFParameters(), 
    phases=phases_Si5, 
    interface=calculator_Si5
)

fitter_Si5 = Fitter(job_Si5, calculator_Si5.fit_func)

#### Set parameters

In [ ]:
parameters5 = job_Si5.parameters

parameters5.get_parameters()

In [ ]:
# parameters to be refined
parameters5.delta2 = df_case5['pdfgui_ini']['delta2'] 

# set values for fixed parameters using metadata in header of .fgr file
parameters5.qmax =  31.4
parameters5.qdamp = 0.024
parameters5.qbroad = 0.02

parameters5.delta1 = 0.0

In [ ]:
phase1_patterns5 = job_Si5.phases[0]
phase1_patterns5.get_parameters()

In [ ]:
phase1_patterns5.cell.length_a = df_case5['pdfgui_ini']['length_a'] 

phase1_patterns5.atoms[0].adp.Uiso = df_case5['pdfgui_ini']['Uiso']

phases_Si5[0].scale = df_case5['pdfgui_ini']['scale']

In [ ]:
# Add initial data used in EasyDiffraction to pandas DataFrame
df_case5['ezd_ini'] = [phases_Si5[0].scale.value, 
                       parameters5.delta2.value, 
                       phase1_patterns5.cell.length_a.value, 
                       phase1_patterns5.atoms[0].adp.Uiso.value]

#### Plot initial profile and compare with observed data

In [ ]:
x_data_Si_20A_01Rmin = data_Si_20A_01Rmin[:, 0]
Gobs_Si_20A_01Rmin = data_Si_20A_01Rmin[:, 1]
Gsim_Si_20A_01Rmin = calculator_Si5.fit_func(x_data_Si_20A_01Rmin)

plot_ini_config(x_data_Si_20A_01Rmin, 
                Gobs_Si_20A_01Rmin, 
                Gsim_Si_20A_01Rmin, 
                r'Si 20 Å 01Rmin to neutron experimental PDF')

#### Select parameters to optimize

For case 5, they are
- scale factor
- delta2
- a=b=c
- uii

In [ ]:
phase1_patterns5.cell.length_a.fixed = False

phase1_patterns5.atoms[0].adp.Uiso.fixed = False

phases_Si5[0].scale.fixed = False

parameters5.delta2.fixed = False

#### Fit

In [ ]:
result_Si5 = fitter_Si5.fit(x_data_Si_20A_01Rmin, 
                            data_Si_20A_01Rmin[:, 1],
                            method='least_squares', 
                            minimizer_kwargs={'diff_step': 1e-5, 'verbose': 2})

#### Print and plot results

In [ ]:
# Add refined values to pandas DataFrame
df_case5['ezd_fit'] = pd.Series(dtype='float64')

for i in range(len(job_Si5.get_fit_parameters())):
    df_case5['ezd_fit'][job_Si5.get_fit_parameters()[i].name] = job_Si5.get_fit_parameters()[i].value
df_case5

In [ ]:
# Plot relative difference of refined values between pdfgui and EasyDiffraction
plot_comparison_refined_params(df_case5, 'Case 5')

In [ ]:
print(f"The fit has been successful: {result_Si5.success}")  
print(f"The goodness of fit (chi2) is: {result_Si5.reduced_chi}")
print("The optimized parameters are:")

for param in job_Si5.get_fit_parameters():
    print(f"{param.name}: {param.value} +\- {param.variance} {param.unit}") 

In [ ]:
# obtain data from PdfFit calculator object
Gfit_Si_20A_01Rmin = calculator_Si1.fit_func(x_data_Si_20A_01Rmin)
Gdiff_Si_20A_01Rmin = Gobs_Si_20A_01Rmin - Gfit_Si_20A_01Rmin

plot_final_config(x_data_Si_20A_01Rmin, 
                  Gobs_Si_20A_01Rmin, 
                  Gsim_Si_20A_01Rmin, 
                  Gfit_Si_20A_01Rmin, 
                  r'Si 20 Å 01Rmin to neutron experimental PDF')